## What are we going to cover
* get  a vision dataset from torchvision.datasets
* architecture of a CNN with PyTorch
* An end to end multi-class image classification problem
* steps in modelling with CNNs in pytorch
    * Creating a CNN model with PyTorch
    * Picking a Loss and optimizer
    * Training a model a model
    * Evaluating a model